In [ ]:
from mindarmour import MembershipInference
import numpy as np

In [ ]:
from mindarmour.utils import LogUtil

In [ ]:
LOGGER = LogUtil.get_instance()
TAG = "MembershipInference_test"
LOGGER.set_level("INFO")

In [2]:

from mindformers import AutoModel, AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

2023-09-21 02:50:32,500 - mindformers[auto_class.py:756] - INFO - default yaml config in ./checkpoint_download/gpt2/gpt2.yaml is used.
2023-09-21 02:50:32,501 - mindformers[auto_class.py:661] - INFO - Config in the yaml file ./checkpoint_download/gpt2/gpt2.yaml are used for tokenizer building.
2023-09-21 02:50:32,518 - mindformers[auto_class.py:668] - INFO - Load the tokenizer name GPT2Tokenizer from the ./checkpoint_download/gpt2/gpt2.yaml
2023-09-21 02:50:32,519 - mindformers[base_tokenizer.py:2042] - INFO - Download the vocab from the url https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/gpt2/vocab.json to ./checkpoint_download/gpt2/vocab.json.
2023-09-21 02:50:32,622 - mindformers[download_tools.py:81] - INFO - Start download ./checkpoint_download/gpt2/vocab.json


Downloading: 1.04MB [00:00, 9.19MB/s]                                                               

2023-09-21 02:50:32,739 - mindformers[download_tools.py:93] - INFO - Download completed!,times: 0.22s
2023-09-21 02:50:32,742 - mindformers[base_tokenizer.py:2042] - INFO - Download the vocab from the url https://ascend-repo-modelzoo.obs.cn-east-2.myhuaweicloud.com/XFormer_for_mindspore/gpt2/merges.txt to ./checkpoint_download/gpt2/merges.txt.


2023-09-21 02:50:32,865 - mindformers[download_tools.py:81] - INFO - Start download ./checkpoint_download/gpt2/merges.txt


Downloading: 457kB [00:00, 4.69MB/s]                                                                

...


[WARNING] ME(1574942:140184214538048,MainProcess):2023-09-21-02:50:33.876.71 [mindspore/ops/primitive.py:228] The in_strategy of the operator in your network will not take effect in stand_alone mode. This means the the shard function called in the network is ignored. 
If you want to enable it, please use semi auto or auto parallel mode by context.set_auto_parallel_context(parallel_mode=ParallelMode.SEMI_AUTO_PARALLEL or context.set_auto_parallel_context(parallel_mode=ParallelMode.AUTO_PARALLEL)
[WARNING] ME(1574942:140184214538048,MainProcess):2023-09-21-02:50:33.172.923 [mindspore/common/parameter.py:778] This interface may be deleted in the future.


2023-09-21 02:50:33,680 - mindformers[download_tools.py:81] - INFO - Start download ./checkpoint_download/gpt2/gpt2.ckpt


Downloading: 498MB [00:21, 23.0MB/s]                                                                

2023-09-21 02:50:55,290 - mindformers[download_tools.py:93] - INFO - Download completed!,times: 21.74s
2023-09-21 02:50:55,294 - mindformers[base_model.py:117] - INFO - start to read the ckpt file: 497772028


2023-09-21 02:50:56,262 - mindformers[base_model.py:121] - INFO - weights in ./checkpoint_download/gpt2/gpt2.ckpt are loaded
2023-09-21 02:50:56,267 - mindformers[auto_class.py:418] - INFO - model built successfully!


In [ ]:

from mindspore import load_checkpoint,load_param_into_net

In [ ]:

params_dict=load_checkpoint('./output/checkpoint/rank_0/gpt_rank_0-1170_1.ckpt')

In [ ]:

_=load_param_into_net(model,params_dict)

In [ ]:
config = [
        {
            "method": "lr",
            "params": {
                "C": np.logspace(-4, 2, 10)
            }
        },
     {
            "method": "knn",
            "params": {
                "n_neighbors": [3, 5, 7]
            }
        },
        {
            "method": "mlp",
            "params": {
                "hidden_layer_sizes": [(64,), (32, 32)],
                "solver": ["adam"],
                "alpha": [0.0001, 0.001, 0.01]
            }
        },
        {
            "method": "rf",
            "params": {
                "n_estimators": [100],
                "max_features": ["auto", "sqrt"],
                "max_depth": [5, 10, 20, None],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4]
            }
        }
    ]

In [ ]:
metrics = ["precision", "accuracy", "recall"]

In [ ]:
import mindspore.nn as nn
from mindspore.common import initializer as init
from mindspore.common.initializer import initializer
from mindspore.train import Model
import mindspore.dataset as ds

In [ ]:
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
opt = nn.Momentum(params=model.trainable_params(), learning_rate=0.1, momentum=0.9,
                  weight_decay=5e-4, loss_scale=1.0)

In [ ]:
base_model=Model(network=model,loss_fn=loss,optimizer=opt)

In [19]:
inference = MembershipInference(base_model) 